In [67]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
data = pd.read_csv('data/cards.csv')

C:\Users\Przemyslaw\.conda\envs\mtg\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,9,12,16,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [80]:
pd.options.display.max_columns = data.shape[1]

In [83]:
data.describe(include='all')

,index,id,artist,asciiName,borderColor,colorIdentity,colorIndicator,colors,convertedManaCost,duelDeck,edhrecRank,faceConvertedManaCost,flavorName,flavorText,frameEffect,frameEffects,frameVersion,hand,hasFoil,hasNoDeckLimit,hasNonFoil,isAlternative,isArena,isBuyABox,isDateStamped,isFullArt,isMtgo,isOnlineOnly,isOversized,isPaper,isPromo,isReprint,isReserved,isStarter,isStorySpotlight,isTextless,isTimeshifted,layout,leadershipSkills,life,loyalty,manaCost,mcmId,mcmMetaId,mtgArenaId,mtgoFoilId,mtgoId,multiverseId,name,names,number,originalText,originalType,otherFaceIds,power,printings,purchaseUrls,rarity,scryfallId,scryfallIllustrationId,scryfallOracleId,setCode,side,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark
count,51430.000000,51430.000000,51428,66,51430,45778,159,40120,51430.000000,1594,46925.000000,877.00000,20,28473,1899,1899,51430,118.000000,51430.000000,51430.000000,51430.000000,51430.00000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430.000000,51430,3000,118.000000,700,44853,38204.000000,38204.000000,3937.000000,24360.000000,30247.000000,38886.000000,51430,965,51430,38084,38873,965,23575,51430,41133,51430,51430,51071,51430,51430,955,31227,6638,44412.000000,50516,23575,51430,51430,51430,6431,4259
unique,NaN,NaN,897,26,5,31,9,39,NaN,2,NaN,NaN,20,16861,15,26,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,3,NaN,16,691,NaN,NaN,NaN,NaN,NaN,NaN,21045,249,6559,26410,2858,965,33,5531,40662,4,50953,25440,20781,467,3,1458,8,NaN,19628,37,1912,33,51430,6431,159
top,NaN,NaN,John Avon,El-Hajjaj,black,G,R,G,NaN,a,NaN,NaN,"Ghidorah, King of the Cosmos","Without the interfering hands of civilization,...",legendary,legendary,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,normal,"{'brawl': False, 'commander': True, 'oathbreak...",NaN,5,{1}{W},NaN,NaN,NaN,NaN,NaN,NaN,Forest,"Fire,Ice",1,G,Instant,"1872a263-5e8e-5cc6-ae58-28fb63c45084,d95cda95-...",2,"10E,2ED,3ED,4BB,4ED,5ED,6ED,7ED,8ED,9ED,AKH,AL...",{'cardmarket': 'https://mtgjson.com/links/9206...,common,8987644d-5a31-4a4e-9a8a-3d6260ed0fd6,78034c44-88d3-47ab-8cc5-2c211c5ded76,b34bb2dc-c1af-4d77-b0b3-a0fb342a5fc6,MB1,b,Aura,Legendary,NaN,({T}: Add {G}.),2,Instant,Creature,a40be3ec-f833-5d1c-aebf-8603f8fa068a,"4b0ded9e-db11-53b7-b733-0e3997933a8d,cde4ee90-...",mirran
freq,NaN,NaN,1075,7,44457,7994,39,7310,NaN,804,NaN,NaN,1,20,717,671,13400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49961,2295,NaN,169,1500,NaN,NaN,NaN,NaN,NaN,NaN,642,22,346,363,4409,1,7053,642,5,19251,5,34,642,1700,474,2207,3432,NaN,647,5834,6028,21823,1,1,348
mean,25714.500000,25715.500000,NaN,NaN,NaN,NaN,NaN,NaN,41.824178,NaN,8259.249462,2.32041,NaN,NaN,NaN,NaN,NaN,0.084746,0.593253,0.000194,0.905717,0.00455,0.068909,0.001322,0.030721,0.007953,0.609722,0.074412,0.007700,0.887128,0.098872,0.578728,0.025919,0.274373,0.001614,0.001983,0.004103,NaN,NaN,1.135593,NaN,NaN,171711.050649,85372.130222,67745.773432,36910.388054,42044.965782,240077.594147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83002.825970,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,14846.706509,14846.706509,NaN,NaN,NaN,NaN,NaN,NaN,6235.930407,NaN,5970.205347,1.75463,NaN,NaN,NaN,NaN,NaN,1.209480,0.491232,0.013943,0.292226,0.06730,0.253302,0.036338,0.172563,0.088823,0.487817,0.262442,0.087411,0.316439,0.298493,0.493768,0.158894,0.446202,0.040141,0.044490,0.063921,NaN,NaN,5.679345,NaN,NaN,152975.960356,107841.046084,2134.511279,17398.227539,20213.679846,178426.440084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70133.509127,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1.000000,0.00000,NaN,NaN,NaN,NaN,NaN,-4.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [25]:
data.head()

,index,id,artist,asciiName,borderColor,colorIdentity,colorIndicator,colors,convertedManaCost,duelDeck,...,subtypes,supertypes,tcgplayerProductId,text,toughness,type,types,uuid,variations,watermark
0,0,1,Rebecca Guay,NaN,black,G,NaN,G,4.0,NaN,...,NaN,NaN,15023.0,"If you would draw a card, you may instead choo...",NaN,Enchantment,Enchantment,1669af17-d287-5094-b005-4b143441442f,NaN,NaN
1,1,2,Stephen Daniele,NaN,black,U,NaN,U,3.0,NaN,...,"Human,Wizard",NaN,15024.0,When Academy Researchers enters the battlefiel...,2,Creature — Human Wizard,Creature,047d5499-a21c-5f5c-9679-1599fcaf9815,NaN,NaN
2,2,3,John Avon,NaN,black,"U,W",NaN,NaN,0.0,NaN,...,NaN,NaN,15025.0,{T}: Add {C}.\n{T}: Add {W} or {U}. Adarkar Wa...,NaN,Land,Land,ee19938c-4007-58f1-8904-fae28007b422,NaN,NaN
3,3,4,Roger Raupp,NaN,black,B,NaN,B,3.0,NaN,...,NaN,NaN,15026.0,Target creature gets -1/-1 until end of turn.\...,NaN,Instant,Instant,8774e18f-3752-5c06-af94-5da3960da9ed,NaN,NaN
4,4,5,Christopher Moeller,NaN,black,G,NaN,G,2.0,NaN,...,NaN,NaN,15027.0,Target creature gets +1/+1 until end of turn.\...,NaN,Instant,Instant,4e875bca-0c52-5d60-889d-1db67e261737,NaN,NaN


In [26]:
data.columns

Index(['index', 'id', 'artist', 'asciiName', 'borderColor', 'colorIdentity',
       'colorIndicator', 'colors', 'convertedManaCost', 'duelDeck',
       'edhrecRank', 'faceConvertedManaCost', 'flavorName', 'flavorText',
       'frameEffect', 'frameEffects', 'frameVersion', 'hand', 'hasFoil',
       'hasNoDeckLimit', 'hasNonFoil', 'isAlternative', 'isArena', 'isBuyABox',
       'isDateStamped', 'isFullArt', 'isMtgo', 'isOnlineOnly', 'isOversized',
       'isPaper', 'isPromo', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'layout',
       'leadershipSkills', 'life', 'loyalty', 'manaCost', 'mcmId', 'mcmMetaId',
       'mtgArenaId', 'mtgoFoilId', 'mtgoId', 'multiverseId', 'name', 'names',
       'number', 'originalText', 'originalType', 'otherFaceIds', 'power',
       'printings', 'purchaseUrls', 'rarity', 'scryfallId',
       'scryfallIllustrationId', 'scryfallOracleId', 'setCode', 'side',
       'subtypes', 'supertypes', 'tcgplayerProduc

In [48]:
data['text']

0        If you would draw a card, you may instead choo...
1        When Academy Researchers enters the battlefiel...
2        {T}: Add {C}.\n{T}: Add {W} or {U}. Adarkar Wa...
3        Target creature gets -1/-1 until end of turn.\...
4        Target creature gets +1/+1 until end of turn.\...
                               ...                        
51425    Two target creatures you control each get +2/+...
51426    Flying\nLandfall — Whenever a land enters the ...
51427    At the beginning of your upkeep, you may choos...
51428    Landfall — Whenever a land enters the battlefi...
51429    Forestwalk (This creature can't be blocked as ...
Name: text, Length: 51430, dtype: object

In [88]:
data_filtered = data[['asciiName', 'text', 'manaCost', 'power', 'life', 'rarity']]

In [57]:
data_filtered

,asciiName,text,manaCost,power,life,rarity
0,NaN,"If you would draw a card, you may instead choo...",{2}{G}{G},NaN,NaN,rare
1,NaN,When Academy Researchers enters the battlefiel...,{1}{U}{U},2,NaN,uncommon
2,NaN,{T}: Add {C}.\n{T}: Add {W} or {U}. Adarkar Wa...,NaN,NaN,NaN,rare
3,NaN,Target creature gets -1/-1 until end of turn.\...,{2}{B},NaN,NaN,common
4,NaN,Target creature gets +1/+1 until end of turn.\...,{1}{G},NaN,NaN,common
...,...,...,...,...,...,...
51425,NaN,Two target creatures you control each get +2/+...,{2}{W}{W},NaN,NaN,uncommon
51426,NaN,Flying\nLandfall — Whenever a land enters the ...,{3}{U},2,NaN,common
51427,NaN,"At the beginning of your upkeep, you may choos...",{3}{W}{W},4,NaN,rare
51428,NaN,Landfall — Whenever a land enters the battlefi...,{1}{R},NaN,NaN,common


### Zamiania typu na string dla tokenizera

In [87]:
data_filtered['text'] = data_filtered['text'].astype(str)

C:\Users\Przemyslaw\.conda\envs\mtg\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Tokenizacja

In [62]:
tokenizer = Tokenizer(oov_token='<OOV>') #upewnić się, że num_words samo się dopasowuje
tokenizer.fit_on_texts(data_filtered['text'])

In [63]:
tokenizer.word_index

{'<OOV>': 1,
 'the': 2,
 'creature': 3,
 'you': 4,
 'a': 5,
 'of': 6,
 '1': 7,
 'your': 8,
 'target': 9,
 'to': 10,
 'this': 11,
 'card': 12,
 'it': 13,
 'battlefield': 14,
 't': 15,
 'or': 16,
 '2': 17,
 'turn': 18,
 'control': 19,
 'and': 20,
 'that': 21,
 'damage': 22,
 'enters': 23,
 'if': 24,
 'may': 25,
 'put': 26,
 'player': 27,
 'with': 28,
 'from': 29,
 'end': 30,
 'each': 31,
 'on': 32,
 'when': 33,
 'add': 34,
 'creatures': 35,
 'library': 36,
 'until': 37,
 'as': 38,
 'whenever': 39,
 'cast': 40,
 'flying': 41,
 'for': 42,
 'cards': 43,
 'deals': 44,
 'its': 45,
 'hand': 46,
 'spell': 47,
 'an': 48,
 'graveyard': 49,
 'any': 50,
 'at': 51,
 '3': 52,
 'counter': 53,
 "can't": 54,
 'gets': 55,
 'life': 56,
 'be': 57,
 'sacrifice': 58,
 'x': 59,
 'draw': 60,
 'mana': 61,
 'then': 62,
 'b': 63,
 'cost': 64,
 'r': 65,
 'is': 66,
 'g': 67,
 'beginning': 68,
 'exile': 69,
 'land': 70,
 'into': 71,
 'u': 72,
 'two': 73,
 'destroy': 74,
 'all': 75,
 'return': 76,
 'in': 77,
 'w': 78

In [65]:
sequences = tokenizer.texts_to_sequences(data_filtered['text'])

### Padding

In [86]:
padded = pad_sequences(sequences)

array([[   0,    0,    0, ...,   77,   50,  199],
       [   0,    0,    0, ...,   10, 1614, 4247],
       [   0,    0,    0, ...,   22,   10,    4],
       ...,
       [   0,    0,    0, ...,    6,   21,  200],
       [   0,    0,    0, ...,  110,   30,  111],
       [   0,    0,    0, ...,  115,    5,  245]])

In [89]:
data_filtered['textTokenized'] = padded

ValueError: Wrong number of items passed 128, placement implies 1

In [85]:
data_filtered

,asciiName,text,manaCost,power,life,rarity
0,NaN,0,{2}{G}{G},NaN,NaN,rare
1,NaN,0,{1}{U}{U},2,NaN,uncommon
2,NaN,0,NaN,NaN,NaN,rare
3,NaN,0,{2}{B},NaN,NaN,common
4,NaN,0,{1}{G},NaN,NaN,common
...,...,...,...,...,...,...
51425,NaN,0,{2}{W}{W},NaN,NaN,uncommon
51426,NaN,0,{3}{U},2,NaN,common
51427,NaN,0,{3}{W}{W},4,NaN,rare
51428,NaN,0,{1}{R},NaN,NaN,common
